In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import time
# Import API key
from api_keys import g_key


In [2]:
readweather_df = pd.read_csv("weather_df.csv")
readweather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Gore,100,NZ,1578602271,73,-46.19,168.86,57.00,13.00
1,1,Ushuaia,75,AR,1578602273,87,-54.81,-68.31,42.80,38.03
2,2,Atuona,20,PF,1578602275,76,-9.80,-139.03,81.97,21.68
3,3,Luganville,75,VU,1578602277,100,-15.51,167.18,77.00,4.70
4,4,San Ramon,75,NI,1578602279,94,12.92,-85.84,69.80,13.87


In [3]:
del readweather_df['Unnamed: 0']
readweather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Gore,100,NZ,1578602271,73,-46.19,168.86,57.00,13.00
1,Ushuaia,75,AR,1578602273,87,-54.81,-68.31,42.80,38.03
2,Atuona,20,PF,1578602275,76,-9.80,-139.03,81.97,21.68
3,Luganville,75,VU,1578602277,100,-15.51,167.18,77.00,4.70
4,San Ramon,75,NI,1578602279,94,12.92,-85.84,69.80,13.87


In [4]:
locations = readweather_df[["Lat","Lng"]]
humidity = readweather_df["Humidity"].astype(float)
maxhumidity = humidity.max()

In [6]:
fig = gmaps.figure(center=(0, 0), zoom_level=3)
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=maxhumidity,
                                 point_radius = 1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
narrow_cities = readweather_df.copy()
narrow_cities = narrow_cities[(narrow_cities["Max Temp"] >70)]
narrow_cities = narrow_cities[(narrow_cities["Cloudiness"] <60)]
narrow_cities = narrow_cities[(narrow_cities["Wind Speed"] <20)]
narrow_cities = narrow_cities.dropna(how='any')
narrow_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,Don Luan,59,VN,1578602290,73,11.53,106.92,70.50,3.15
11,Maragogi,0,BR,1578602054,72,-9.01,-35.22,80.87,13.94
15,Hithadhoo,59,MV,1578602302,78,-0.60,73.08,82.45,13.94
21,Rio Grande,40,BR,1578602152,74,-32.03,-52.10,82.40,8.05
33,Hirna,0,ET,1578602340,58,9.29,41.09,70.74,6.44


## Hotel Map 

In [8]:
hotel_df = narrow_cities
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,Don Luan,59,VN,1578602290,73,11.53,106.92,70.50,3.15
11,Maragogi,0,BR,1578602054,72,-9.01,-35.22,80.87,13.94
15,Hithadhoo,59,MV,1578602302,78,-0.60,73.08,82.45,13.94
21,Rio Grande,40,BR,1578602152,74,-32.03,-52.10,82.40,8.05
33,Hirna,0,ET,1578602340,58,9.29,41.09,70.74,6.44


In [9]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
9,Don Luan,59,VN,1578602290,73,11.53,106.92,70.50,3.15,
11,Maragogi,0,BR,1578602054,72,-9.01,-35.22,80.87,13.94,
15,Hithadhoo,59,MV,1578602302,78,-0.60,73.08,82.45,13.94,
21,Rio Grande,40,BR,1578602152,74,-32.03,-52.10,82.40,8.05,
33,Hirna,0,ET,1578602340,58,9.29,41.09,70.74,6.44,


In [ ]:

target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    params["location"]=(f"{latitude},{longitude}")

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
hotel_response = response.json()

try:
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        print(f"Hotel name is {hotel_response}")
except (IndexError, KeyError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print(f"No hotel found in {row['City']}")

        time.sleep(1.50)
#print(response.url)

In [10]:
hotel_df.sort_values(by="City")

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
110,Acapulco de Juarez,5,MX,1578602501,70,16.86,-99.88,86.00,9.17,
257,Alibag,9,IN,1578602812,56,18.64,72.88,73.40,5.82,
450,Alta Floresta,40,BR,1578603219,78,-9.87,-56.08,80.60,2.24,
428,Am Timan,15,TD,1578603173,30,11.04,20.28,74.48,4.07,
465,Anloga,20,GH,1578603251,89,5.79,0.90,84.20,6.93,
...,...,...,...,...,...,...,...,...,...,...
126,Torquay,27,AU,1578602535,88,-25.30,152.85,75.99,6.93,
147,Tres Arroyos,0,AR,1578602580,23,-38.31,-60.23,77.50,12.95,
105,Tuxpan,20,MX,1578602491,45,19.56,-103.38,84.20,5.37,
434,Yulara,40,AU,1578603125,61,-25.24,130.99,82.40,4.70,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]